In [1]:
import pandas as pd

In [2]:
import cx_Oracle

def hero_to_df(Userid, Password, Host, Port, SID, query):

    dsn_tns = cx_Oracle.makedsn(host=Host, port=Port, sid=sid_name)
    conn = cx_Oracle.connect(user=Userid, password=Password, dsn=dsn_tns)

    df = pd.read_sql_query(query, conn)
    conn.close()
    
    return(df)

## HEROPRD
Userid = "rrice"
Password = "Temp02122020" 
Host = "herodb.rtpnc.epa.gov"
Port = "1521"
sid_name = "heroprd"

def defaultquery(query):
    return hero_to_df(Userid, Password, Host, Port, sid_name, query)

query = '''select reference_id, year, title 
    from tbl_reference 
    where sdelete = 'No' 
    and reference_id in
    (select reference_id from tbl_reference_usage where usage_id = 19134 and isdeleted = 0)'''

df = defaultquery(query)
df.head()

,REFERENCE_ID,YEAR,TITLE
0,4310874,2018.0,A new framework of global sensitivity analysis...
1,3399227,2015.0,Accumulation of secondary metabolites in healt...
2,3399876,2013.0,Contrasting Physiological Responses of Jatroph...
3,3399878,2013.0,Gene Knockout Study Reveals That Cytosolic Asc...
4,3400890,2013.0,Ascorbate as seen through plant evolution: the...


In [3]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def nltk_process(words):

    tokens = word_tokenize(words)
    stopword_filtered = [w for w in tokens if not w in stop_words]
    
    stemmed_words = []
    for word in stopword_filtered:
        stemmed_words.append(ps.stem(word))
    
    #remove digits and punctuation   
    pattern = re.compile(r'[a-zA-Z]+')
    cleaned = [x for x in stemmed_words if pattern.search(x)]
    
    return cleaned
    
nltkdf = pd.DataFrame()
nltkdf['Text'] = df['TITLE'].dropna()  ###########changed to abstract
nltkdf['Text_Processed'] = nltkdf['Text'].apply(lambda x: nltk_process(x))

In [4]:
nltkdf.join(df).iloc[:,:].to_pickle('nltk_stemmed.pkl')